# Example 3-2: Finding Station Coordinates
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 146

This notebook demonstrates finding the geodetic and geocentric position vectors for a site given its geodetic latitude, longitude, and height.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install -r valladopy_version.txt

Then import `numpy` and the relevant modules from `valladopy`:

In [2]:
import numpy as np
import valladopy.constants as const
from valladopy.astro.twobody.utils import site, gd2gc
from valladopy.mathtime.utils import dms2rad

## Problem Definition
---

GIVEN: Ascension Island coordinates: $\phi_{gd} = -7^\circ\ 54'\ 23.886^{\prime\prime}$, $\lambda = 345^\circ\ 35'\ 51.000^{\prime\prime}$, $h_{ellp} = 56$ m<br> 
FIND: Position vectors

In [3]:
latgd_dms = (-7, -54, -23.886)  # geodetic latitude, (deg, min, sec)
lon_dms = (345, 35, 51)         # longitude, (deg, min, sec)
alt = 0.056                     # altitude, km

## Solution
---

First, we must find the decimal latitude and longitude values by converting units. We can leverage the `dms2rad` routine (an implementation of **Algorithm 17**) to accomplish this:

In [4]:
latgd = dms2rad(*latgd_dms)  # geodetic latitude, rad
lon = dms2rad(*lon_dms)      # longitude, rad

print(f'latgd:\t{np.degrees(latgd):.4f}\t\tdeg')
print(f'lon:\t{np.degrees(lon):.4f}\tdeg')

latgd:	-7.9066		deg
lon:	345.5975	deg


Using **Eq. 3-7**, we can find the two auxiliary quantities that are obtained from geometrical properties of an ellipse:


$$
C_{\oplus} = \frac{R_{\oplus}}{\sqrt{1 - e_{\oplus}^2\sin^2\phi_{gd}}}, \quad S_{\oplus} = \frac{R_{\oplus}(1-e_{\oplus}^2)}{\sqrt{1 - e_{\oplus}^2\sin^2\phi_{gd}}}
$$

In [5]:
cearth = const.RE / np.sqrt(1 - const.ECCEARTHSQRD * np.sin(latgd) ** 2)
searth = cearth * (1 - const.ECCEARTHSQRD)

print(f'cearth:\t{cearth:.4f} km')
print(f'searth:\t{searth:.4f} km')

cearth:	6378.5403 km
searth:	6335.8399 km


The geodetic position vector can then be found using **Eq. 3-14**:

$$
\vec{r}_{IJK} = \begin{bmatrix}
(C_{\oplus} + h_{ellp}) \cos(\phi_{gd}) \cos{\lambda} \\
(C_{\oplus} + h_{ellp}) \cos(\phi_{gd}) \sin{\lambda} \\
(S_{\oplus} + h_{ellp}) \sin(\phi_{gd})
\end{bmatrix}
$$

We can leverage the `site` routine to calculate this:

In [6]:
r_gd, _ = site(latgd, lon, alt)

print(f'r (geodetic):\t{r_gd} km')

r (geodetic):	[ 6119.39959773 -1571.47938298  -871.56108524] km


To get the geocentric position vector, we first convert the geodetic latitude to geocentric latitude using **Eq. 3-11**:

$$
\tan(\phi_{gc}) = (1 - e_{\oplus}^2) \tan(\phi_{gd})
$$

Use the `gd2gc` routine for the conversion:

In [7]:
latgc = gd2gc(latgd)  # geocentric latitude, rad

print(f'latgc:\t{np.degrees(latgc):.4f} deg')

latgc:	-7.8544 deg


Finally, use **Eq. 3-14** to find the geocentric position vector:

$$
\vec{r}_{IJK} = \lVert \vec{r_{IJK}} \rVert \begin{bmatrix}
\cos(\phi_{gc}) \cos{\lambda} \\
\cos(\phi_{gc}) \sin{\lambda} \\
\sin(\phi_{gc})
\end{bmatrix}
$$

In [8]:
# Calculate the magitude of the geodetic position vector
rmag = np.linalg.norm(r_gd)

# Construct the geocentric position vector
r_gc = rmag * np.array(
    [
        np.cos(latgc) * np.cos(lon),
        np.cos(latgc) * np.sin(lon),
        np.sin(latgc)
    ]
)

print(f'r (geocentric):\t{r_gc} km')

r (geocentric):	[ 6119.39960449 -1571.47938472  -871.56103464] km


The small difference between the two answers is from truncation errors, and these vectors are actually in an *Earth-fixed* frame.